In [1]:
from typing import List, Tuple
import graphviz
import pandas as pd
import json
import requests

response = requests.get("https://zetsuboushii.github.io/tome-of-the-vastlands/api/characters.json", stream=True)
data = response.json()

relationship_types = ["parent", "parentOf", "maried", "siblings", "killedBy"]

relationship_types = ["relationships"]
data = {"characters": data}

relationships: List[Tuple[str, str, str]] = []


for character in data["characters"]:
    name = character.get("name", "")
    #if name != (character.get("firstname", "") + " " + character.get("lastname", "")):
        #continue
    for rel_type in relationship_types:
        relationships_of_type = character.get(rel_type, [])
        if not isinstance(relationships_of_type, list): # Filter invalid data
            continue
        for rname, relationship_name in relationships_of_type:
            #if len(relationship_name.split(" ")) == 1: # Filter relationship targets with only one name
                #continue
            ignored = ["U-Ranos", "Nayru", "Siegmund"]
            if rname in ignored or name in ignored:
                continue
            relationships.append((name, rname, relationship_name))

rel_df = pd.DataFrame(relationships, columns=["Name", "Relationship", "Type"])
rel_df

,Name,Relationship,Type
0,Adamar,Farone,Mutter/'Kleine Schwester'
1,Adamar,Finnea,Große Schwester
2,Adamar,Phaerille,Kleine Schwester
3,Aelua,Tannivh,Enkel
4,Ar-Merer,Justicia,Mutter
...,...,...,...
430,Zetta,Arvel,verstorbene große Liebe
431,Zetta,Valerius,"langjähriger Freund, Vorbild"
432,Zetta,Carmesine,Freundin
433,Zetta,Felix,Freund


In [2]:
with open("graphviz.dot", "w") as file:
    file.write("digraph G {\n")
    for name_a, name_b, rel_type in relationships:
        file.write("\"" + name_a + "\" -> \"" + name_b + "\" [label=\"" + rel_type + "\"];\n")
    file.write("}")

In [ ]:
graphviz.render("dot", filepath="graphviz.jpg", format="jpg")